In [1]:
import textract
import PySimpleGUI as sg

In [2]:
import os
import glob
import re
import subprocess
from collections import defaultdict
import string
extensions = ['docx', 'pdf', 'pptx']
exclude = set(string.punctuation).difference(set(['?','!', '.', ',', "'"]))


TypeError: set expected at most 1 arguments, got 5

In [3]:
def glob_re(pattern, strings):
    return filter(re.compile(pattern).match, strings)

def process(directory):
    os.chdir(directory)
    if values['docx']:
        for doc in glob.iglob('*.doc'):
                subprocess.call(['soffice', '--headless', '--convert-to', 'docx', doc])#'--outdir',directory + "/temp", doc])
                print("Converted old {} to new {}x".format(doc, doc))
                os.remove(doc)
                print("Deleted old {}".format(doc))
                

def get_all_subfolders(directory):
    os.chdir(directory) 

    # Mark all the vertices as not visited 
    visited = defaultdict(lambda:False) 

    # Create a queue for BFS 
    queue = [] 

    # Mark the source node as  
    # visited and enqueue it 
    queue.append(directory) 
    visited[directory] = True

    while queue: 

        # Dequeue a vertex from  
        # queue and print it 
        s = queue.pop(0) 
        print (s, end = " ") 
        os.chdir(s)
        # Get all adjacent vertices of the 
        # dequeued vertex s. If a adjacent 
        # has not been visited, then mark it 
        # visited and enqueue it 
        for i in glob.glob('*/'): 
            if visited[i] == False: 
                queue.append(s + "/"+ i) 
                visited[i] = True
    
    return visited

In [6]:
layout = [[sg.Text('Browse to RESPECTIVE directory')],    
                 [sg.Text('IN-DIR', size=(8, 1)), sg.Input(key ='input'), sg.FolderBrowse()],      
                 [sg.Text('OUT-FILE', size=(8, 1)), sg.Input(key ='output'), sg.FileBrowse()],
                 [sg.Frame(layout=[ 
                                    [sg.Checkbox('*.doc and *.docx', size=(20,1), default= True, key = 'docx')],
                                   # [sg.Checkbox('*.txt', size=(20,1), default= True, key = 'txt')],
                                    [sg.Checkbox('*.pptx', size=(10,1), default= True, key = 'pptx'),sg.Checkbox('*.pdf', size=(10,1), default= True, key = 'pdf')]], title = 'Input FileTypes')],                 
              [sg.Checkbox('clean non word charcters?', size=(20,1), default= False, key = 'clean')],   
              [sg.Submit(), sg.Cancel()]]

sg.ChangeLookAndFeel('BrownBlue')
window = sg.Window('Rahul\'s Custom TextExtractor', layout)  

event, values = window.Read()  

  
p = "|".join([extension for extension in extensions if values[extension]])
p = r".*(.{})$".format(p)

print(p)

if event == 'Submit' or event == None :
    directories = get_all_subfolders(values['input'])
    if event:
        for directory in directories:
            process(directory)
            filenames = list(glob_re(p, os.listdir()))
            print("Will Extract Text from {} files:\n{}\n".format(len( filenames), filenames) )
            for doc in filenames:
                with open(values['output'], 'ab') as f:
                    f.write(textract.process(doc))
        
        print("Extraction Complete")
        with open(values['output'], 'r+') as f:
            data = f.read()
        
        print("PostProcessing Started")
        data = re.sub("\s\s+", " ", data)
        
        if values['clean']:
            data = data.translate(str.maketrans('', '', ))

        print("PostProcessing Completed writing To File")
        with open(values['output'], 'w+') as f:
            f.write(data)
            
        print("Process complete checkout file {}".format(values['output']))
    
    window.Close()

.*(.)$


TypeError: chdir: path should be string, bytes, os.PathLike or integer, not NoneType